In [2]:
import java.util.stream.*
import java.util.concurrent.*

In [3]:
//cpu核心数
println(Runtime.getRuntime().availableProcessors())
//最大线程数
println(ForkJoinPool.getCommonPoolParallelism())
//当前线程数
println(ForkJoinPool.commonPool().poolSize)

8
7
0


## 多个任务全部完成后，打印总体时间

In [36]:
val dished = IntStream.range(0,7)
    .mapToObj({
        CompletableFuture.runAsync(
                {
                    println(it.toString()+":"+Thread.currentThread().name+":"+System.currentTimeMillis())
                    Thread.sleep(2000)
                }
            )
    })
    .toList()

val start = System.currentTimeMillis()
CompletableFuture.allOf(*dished.toTypedArray()).join()
println(System.currentTimeMillis() - start)

0:ForkJoinPool.commonPool-worker-24:1662208665500
1:ForkJoinPool.commonPool-worker-25:1662208665525
6:ForkJoinPool.commonPool-worker-30:1662208665540
5:ForkJoinPool.commonPool-worker-29:1662208665539
4:ForkJoinPool.commonPool-worker-28:1662208665538
3:ForkJoinPool.commonPool-worker-27:1662208665533
2:ForkJoinPool.commonPool-worker-26:1662208665530
2120


## 异步执行一个任务（不阻塞笔记本）

In [32]:
val log = java.util.StringJoiner("\n")
CompletableFuture.runAsync({
    log.add("开始睡眠")
    Thread.sleep(5000)
    log.add("结束睡眠")
}).whenCompleteAsync({a,b->
    println(log.toString())
})

java.util.concurrent.CompletableFuture@1ddd3478[Not completed]

In [33]:
log

开始睡眠
结束睡眠

## 异步执行一个任务，并等待输出（阻塞笔记本）

In [31]:
CompletableFuture.runAsync({
    println("开始睡眠")
    Thread.sleep(5000)
    println("结束睡眠")
}).whenCompleteAsync({a,b->
    println(a)
    println(b)
}).join()

开始睡眠
结束睡眠
null
null


null

In [13]:
@file:DependsOn("org.jetbrains.kotlinx:kotlinx-coroutines-core:1.6.4")
import kotlinx.coroutines.*

In [21]:
println("===")
runBlocking {
    launch { doWorld() }
    println("Hello")
}

suspend fun doWorld() {
    delay(1000L)
    println("World!")
}
println("===")

===
Hello
World!
===


## 直接开启一个线程执行任务，不阻塞笔记本

In [21]:
println(Thread.currentThread().name)
val log = java.util.StringJoiner("\n")
val t = java.lang.Thread({
    log.add("hello "+Thread.currentThread().name)
    Thread.sleep(10000)
    log.add("world "+Thread.currentThread().name)
})
t.start()
println(t.state)

Thread-99
RUNNABLE


In [22]:
println(log)
println(t.state)
println("======")
Thread.sleep(10000)
println(log)
println(t.state)

hello Thread-100
TIMED_WAITING
hello Thread-100
world Thread-100
TERMINATED


## 如果开多个线程去跑任务，怎么align所有线程状态？

In [ ]:
CompletableFuture.allOf(*list.toTypedArray()).join()

## 如果线程池满了，拒绝任务

8个核心线程，队列容量8，8个备用线程（即最多16个线程），线程保留8s

In [28]:
fun runCount(count:Int){
    //线程满了在当前线程执行
    val rejectHandler = object: RejectedExecutionHandler {
                        override fun rejectedExecution(r:Runnable, executor:ThreadPoolExecutor){
                            println(String.format("任务被拒绝 %s %s %s", r,executor.taskCount,executor.activeCount));
                            r.run();
                        }
                    }
    val executor = ThreadPoolExecutor(
                    8, 16,
                    10L, TimeUnit.SECONDS,
                    LinkedBlockingQueue<Runnable>(8),
                    rejectHandler);
    val list = mutableListOf<CompletableFuture<Void>>()
    for(i in 1..count){
        //在自定义线程池跑任务
        val cf = CompletableFuture.runAsync({
            println(Thread.currentThread().getName()+"-in-"+i)
            Thread.sleep(5000)
        },executor)
        list.add(cf)
    }
    //睡眠避免日志错乱
    Thread.sleep(500)
    println("等待所有完成开始")
    CompletableFuture.allOf(*list.toTypedArray()).join()
    println("等待所有完成结束")
    executor.shutdown()
}

In [34]:
//核心线程拉满
//队列未满
runCount(15)

pool-24-thread-1-in-1
pool-24-thread-2-in-2
pool-24-thread-4-in-4
pool-24-thread-3-in-3
pool-24-thread-6-in-6
pool-24-thread-8-in-8
pool-24-thread-5-in-5
pool-24-thread-7-in-7
等待所有完成开始
pool-24-thread-1-in-11
pool-24-thread-3-in-15
pool-24-thread-8-in-14
pool-24-thread-6-in-13
pool-24-thread-5-in-12
pool-24-thread-4-in-9
pool-24-thread-2-in-10
等待所有完成结束


In [35]:
//核心线程拉满
//队列拉满
runCount(16)

pool-25-thread-1-in-1
pool-25-thread-2-in-2
pool-25-thread-3-in-3
pool-25-thread-4-in-4
pool-25-thread-6-in-6
pool-25-thread-5-in-5
pool-25-thread-8-in-8
pool-25-thread-7-in-7
等待所有完成开始
pool-25-thread-4-in-9
pool-25-thread-2-in-12
pool-25-thread-3-in-11
pool-25-thread-1-in-10
pool-25-thread-6-in-15
pool-25-thread-7-in-13
pool-25-thread-5-in-14
pool-25-thread-8-in-16
等待所有完成结束


In [36]:
//核心线程拉满
//队列拉满
//备用线程未满（备用线程启动时先执行队列满了之后的任务，即17-20）
runCount(20)

pool-26-thread-1-in-1
pool-26-thread-4-in-4
pool-26-thread-5-in-5
pool-26-thread-6-in-6
pool-26-thread-3-in-3
pool-26-thread-2-in-2
pool-26-thread-12-in-20
pool-26-thread-11-in-19
pool-26-thread-10-in-18
pool-26-thread-9-in-17
pool-26-thread-8-in-8
pool-26-thread-7-in-7
等待所有完成开始
pool-26-thread-4-in-9
pool-26-thread-12-in-14
pool-26-thread-3-in-16
pool-26-thread-2-in-15
pool-26-thread-11-in-13
pool-26-thread-1-in-12
pool-26-thread-5-in-11
pool-26-thread-6-in-10
等待所有完成结束


In [37]:
//核心线程拉满
//队列拉满
//备用线程拉满
runCount(24)

pool-27-thread-1-in-1
pool-27-thread-4-in-4
pool-27-thread-3-in-3
pool-27-thread-2-in-2
pool-27-thread-7-in-7
pool-27-thread-6-in-6
pool-27-thread-5-in-5
pool-27-thread-16-in-24
pool-27-thread-15-in-23
pool-27-thread-14-in-22
pool-27-thread-13-in-21
pool-27-thread-12-in-20
pool-27-thread-11-in-19
pool-27-thread-10-in-18
pool-27-thread-9-in-17
pool-27-thread-8-in-8
等待所有完成开始
pool-27-thread-1-in-10
pool-27-thread-4-in-11
pool-27-thread-3-in-12
pool-27-thread-2-in-9
pool-27-thread-13-in-13
pool-27-thread-15-in-16
pool-27-thread-16-in-15
pool-27-thread-14-in-14
等待所有完成结束


In [38]:
//核心线程拉满
//队列拉满
//备用线程拉满
//任务被拒绝
runCount(28)

pool-28-thread-1-in-1
pool-28-thread-3-in-3
pool-28-thread-2-in-2
pool-28-thread-5-in-5
pool-28-thread-4-in-4
pool-28-thread-16-in-24
pool-28-thread-15-in-23
pool-28-thread-14-in-22
pool-28-thread-13-in-21
pool-28-thread-12-in-20
pool-28-thread-11-in-19
pool-28-thread-10-in-18
pool-28-thread-9-in-17
pool-28-thread-8-in-8
pool-28-thread-7-in-7
任务被拒绝 java.util.concurrent.CompletableFuture$AsyncRun@156a899a 24 16
Thread-161-in-25
pool-28-thread-6-in-6
pool-28-thread-5-in-10
pool-28-thread-3-in-12
pool-28-thread-2-in-11
pool-28-thread-1-in-9
pool-28-thread-16-in-13
pool-28-thread-14-in-15
pool-28-thread-4-in-16
pool-28-thread-15-in-14
pool-28-thread-13-in-26
pool-28-thread-7-in-28
pool-28-thread-11-in-27
等待所有完成开始
等待所有完成结束
